In [2]:
from google.colab import drive
import os

drive.mount('/content/drive')

DATASET_PATH = '/content/drive/MyDrive/cvpr_final'

Mounted at /content/drive


In [3]:
from PIL import Image

deleted_count = 0
for subdir, dirs, files in os.walk(DATASET_PATH):
    for file in files:
        filepath = os.path.join(subdir, file)
        try:
            img = Image.open(filepath)
            img.verify()
        except:
            os.remove(filepath)
            deleted_count += 1

print(f"Cleaned up {deleted_count} bad files from local storage.")

Cleaned up 0 bad files from local storage.


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

IMG_SIZE = (150, 150)
BATCH_SIZE = 32

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 1493 images belonging to 97 classes.
Found 354 images belonging to 97 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models, Input
from tensorflow.keras.optimizers import Adam
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_generator, validation_data=val_generator, epochs=20)

model.save('transfer_learning_attendance.h5')
print("Transfer Learning Model Saved")

/tmp/ipython-input-3771296384.py:4: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 372s 8s/step - accuracy: 0.0275 - loss: 5.0632 - val_accuracy: 0.1130 - val_loss: 4.0613
Epoch 2/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 171s 4s/step - accuracy: 0.1163 - loss: 4.1332 - val_accuracy: 0.2260 - val_loss: 3.6285
Epoch 3/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 163s 3s/step - accuracy: 0.2164 - loss: 3.5873 - val_accuracy: 0.3672 - val_loss: 3.1587
Epoch 4/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 163s 3s/step - accuracy: 0.3075 - loss: 3.1528 - val_accuracy: 0.4859 - val_loss: 2.7553
Epoch 5/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 161s 3s/step - accuracy: 0.3736 - loss: 2.7616 - val_accuracy: 0.5847 - val_loss: 2.3520
Epoch 6/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 170s 4s/step - accuracy: 0.4722 - loss: 2.4018 - val_accuracy: 0.6469 - val_loss: 2.0081
Epoch 7/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 160s 3s/step - accuracy: 0.5575 - loss: 2.0361 - val_accuracy: 0.7147 - val_loss: 1.7196
Epoch 8/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 159s 3s/step - accuracy: 0.6045 - loss: 1.8186 - val_accuracy: 0.7627 - v

Transfer Learning Model Saved


In [6]:
labels = list(train_generator.class_indices.keys())
with open("labels.txt", "w") as f:
    f.write("\n".join(labels))

In [7]:
model.save('transfer_learning_attendance.keras')